In [6]:
import subprocess
import pyautogui
import sys
import os
import time
import pytesseract
from PIL import Image
import numpy as np
import numpy.ma as ma
import math
import fakeWebCam

loginID = '725 405 7950'
password = '140287'
thresh = 5 #how many or more people have to leave in 5 seconds to trigger logout

In [ ]:
print('logging into zoom')
result = subprocess.run(['C:/Users/micha/AppData/Roaming/Zoom/bin/Zoom.exe'])
time.sleep(0.5)
joinButton = pyautogui.locateCenterOnScreen(os.getcwd() + '/assets/join.png', grayscale=True, confidence=0.9)
pyautogui.click(joinButton)
time.sleep(0.5)
pyautogui.typewrite(loginID + '\n')
time.sleep(1)
pyautogui.typewrite(password + '\n')
loginButton = pyautogui.locateCenterOnScreen(os.getcwd() + '/assets/login.png', grayscale=True, confidence=0.9)
pyautogui.click(loginButton)

bgFound = None
while bgFound == None:
    bgFound = pyautogui.locateCenterOnScreen(os.getcwd() + '/assets/bg.png', grayscale=True, confidence=1)
    time.sleep(10)
pyautogui.click(bgFound)

logging into zoom


In [2]:
def startVideo():
    try:
        shell = get_ipython().__class__.__name__
    x = threading.Thread(target=fakeWebCam.falseCam, args=(), daemon=True)
    x.start()
    time.sleep(3)
    video = pyautogui.locateOnScreen(os.getcwd() + '/assets/startVideo.png', grayscale=True, confidence=0.9)
    if video == None:
        pyautogui.press('alt')
        video = pyautogui.locateOnScreen(os.getcwd() + '/assets/startVideo.png', grayscale=True, confidence=0.9)
    pyautogui.click(video[0] + video[2], video[1])
    video = pyautogui.locateCenterOnScreen(os.getcwd() + '/assets/camName.png', grayscale=True, confidence=0.9)
    pyautogui.click(video)
    video = pyautogui.locateCenterOnScreen(os.getcwd() + '/assets/startVideo.png', grayscale=True, confidence=0.9)
    pyautogui.click(video)

In [3]:
def breakout():
    #1. enter breakout if available
        #2. start loop to check for leave breakout
    breakout = pyautogui.locateCenterOnScreen(os.getcwd() + '/assets/breakoutJoin.png', grayscale=True, confidence=0.9)
    if breakout:
        print('breakout')
        pyautogui.click(breakout)
        while True():
            time.sleep(30)
            leaveBreakout = pyautogui.locateCenterOnScreen(os.getcwd() + '/assets/breakoutLeave.png', grayscale=True, confidence=0.9)
            if leaveBreakout:
                pyautogui.click(leaveBreakout)
                time.sleep(30)
                return True
    print('no breakout')
    return False

In [4]:
def userCount():
    #1. screenshot # of participants
    #2. turn completely black and white
    #3. save image as cap.png
    #4. use tesseract to turn image to int
    #5. return int value of participants
    participantsLocation = pyautogui.locateCenterOnScreen(os.getcwd() + '/assets/participants.png', grayscale=True, confidence = 0.8)
    if participantsLocation == None:
        pyautogui.press('alt')
        participantsLocation = pyautogui.locateCenterOnScreen(os.getcwd() + '/assets/participants.png', grayscale=True, confidence = 0.8)
    people = pyautogui.screenshot()
    people = people.crop((participantsLocation.x,
                         participantsLocation.y-30,
                         participantsLocation.x + 30,
                         participantsLocation.y))
    
    data = np.asarray(people)
    a = lambda a: int(np.mean(a))
    data = np.apply_along_axis(a, -1, data)
    data[ma.masked_greater(data, 130).mask] = 255
    data[ma.masked_less_equal(data, 130).mask] = 0
    data = 255 - data
    data = data.flatten()
    img = np.array([])
    for i in data:
        img = np.append(img, np.array([i, i, i], dtype=np.int32))
    img = np.reshape(img, (30,30,3))
    img = img.astype(np.uint8)
    people = Image.fromarray(img)
    people = people.resize((300, 300), Image.BILINEAR)
    people.save(os.getcwd() + '/assets/cap.png')
    
    pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'
    text = pytesseract.image_to_string(img, lang='eng', config='--psm 7 digits')
    text = text[:text.index('\n')]
    return int(text)

In [8]:
def leave():
    #click leave and confirm leave
    print('leaving meeting')
    leave = pyautogui.locateCenterOnScreen(os.getcwd() + '/assets/leave.png', grayscale=True, confidence=0.9)
    if leave == None:
        pyautogui.press('alt')
        leave = pyautogui.locateCenterOnScreen(os.getcwd() + '/assets/leave.png', grayscale=True, confidence=0.9)
    pyautogui.click(leave)
    leave = pyautogui.locateCenterOnScreen(os.getcwd() + '/assets/leaveConfirm.png', grayscale=True, confidence=0.9)
    pyautogui.click(leave)

leaving meeting


In [10]:
if __name__ == '__main__':
    startVideo()
    old = None
    while True:
        time.sleep(5)
        if breakout():
            old = None
        count = userCount()
        print(count, 'users')
        if old != None and old - count > thresh:
            print('Users leaving')
            leave()
        old = count

(640, 480)


Exception in thread Thread-877:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\micha\github\ZoomHandler\fakeWebCam.py", line 20, in falseCam
    with pyvirtualcam.Camera(width=width, height=height, fps=fps, delay=50) as cam:
  File "C:\ProgramData\Anaconda3\lib\site-packages\pyvirtualcam\camera.py", line 86, in __init__
    _native_windows.start(width, height, fps, delay)
RuntimeError: error starting virtual camera output



no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
35 users
no breakout
3

AttributeError: 'NoneType' object has no attribute 'x'

Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\micha\github\ZoomHandler\fakeWebCam.py", line 24, in falseCam
    frame[:,:,:3] = cv2.resize (f, (width, height), interpolation=None)
cv2.error: OpenCV(4.5.0) ..\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'




'ZMQInteractiveShell'

In [158]:
'''leave = pyautogui.locateCenterOnScreen(os.getcwd() + '/assets/viewOptions.png', grayscale=True, confidence = 0.9)
pyautogui.click(leave)
time.sleep(0.01)
leave = pyautogui.locateCenterOnScreen(os.getcwd() + '/assets/requestRemoteControl.png', grayscale=True, confidence=0.9)
pyautogui.click(leave)
time.sleep(0.01)
while True:
    leave = pyautogui.locateCenterOnScreen(os.getcwd() + '/assets/request.png', grayscale=True, confidence=0.9)
    if leave:
        pyautogui.click(leave)'''

"leave = pyautogui.locateCenterOnScreen(os.getcwd() + '/assets/viewOptions.png', grayscale=True, confidence = 0.9)\npyautogui.click(leave)\ntime.sleep(0.01)\nleave = pyautogui.locateCenterOnScreen(os.getcwd() + '/assets/requestRemoteControl.png', grayscale=True, confidence=0.9)\npyautogui.click(leave)\ntime.sleep(0.01)\nwhile True:\n    leave = pyautogui.locateCenterOnScreen(os.getcwd() + '/assets/request.png', grayscale=True, confidence=0.9)\n    if leave:\n        pyautogui.click(leave)"

In [ ]:
bgFound = None
while bgFound == None:
    bgFound = pyautogui.locateCenterOnScreen(os.getcwd() + '/assets/bg.png', grayscale=True, confidence=1)
    time.sleep(10)
pyautogui.click(bgFound)